# Exercices Chapitre 1

## SQLite

### 1) Read all the data from bc.csv, how many rows is there ? Columns ? 

In [3]:
import pandas as pd
data = pd.read_csv("C:\\Users\\Ju\\Desktop\\WIRTZ!\\bc.csv")
print("Il y a", len(data.columns), "colonnes et", len(data), "lignes")
data.head()

Il y a 33 colonnes et 569 lignes


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### 2) Insert the bc.csv in the sqlite format.

Importation depuis DBbrowser

### 3) In the same db create a new table called tomatch with two variables id_ and rdm_float.

In [2]:
import sqlite3

conn = sqlite3.connect("C:/Users/Ju/OneDrive/Documents/FSEG/M1/NoSQL/bc.db")
c = conn.cursor()
c.execute("""CREATE TABLE tomatch(id INT PRIMARY KEY, rdm_float FLOAT)""")
conn.commit()
c.close()

### 4) In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.

In [3]:
import tqdm
import random as rd
c = conn.cursor()
query = """INSERT INTO tomatch(id, rdm_float) VALUES (?,?)"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,1)):
    values = (i,rd.random())
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()
c.close()

100%|██████████| 9950000/9950000 [00:59<00:00, 167620.50it/s]


### 5) Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

In [5]:
c = conn.cursor()
c.execute("""SELECT AVG(bc.radius_mean), COUNT(bc.radius_mean), MAX(bc.radius_mean), MIN(bc.radius_mean)
             FROM bc 
             INNER JOIN tomatch ON bc.id = tomatch.id""")
results = c.fetchone()
print(f"Average Value: {results[0]}")
print(f"Count Value: {results[1]}")
print(f"Max Value: {results[2]}")
print(f"Min Value: {results[3]}")
c.close()

Average Value: 14.010595041322313
Count Value: 484
Max Value: 28.11
Min Value: 6.981


### 6) How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

In [6]:
c = conn.cursor()
c.execute("""SELECT COUNT(*) FROM bc WHERE radius_mean > 15""")
results = c.fetchone()
print(f"Observations with radius_mean greater than 15 : {results[0]}")

c.execute("""SELECT COUNT(*) FROM bc WHERE radius_mean > 15 AND texture_mean > 20""")
results = c.fetchone()
print(f"Observations with radius_mean greater than 15 and texture_mean greater thant 20 : {results[0]}")
c.close()

Observations with radius_mean greater than 15 : 173
Observations with radius_mean greater than 15 and texture_mean greater thant 20 : 106


### 7) For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups

In [11]:
c = conn.cursor()
c.execute("""SELECT diagnosis, ROUND(AVG(radius_mean),2) FROM bc GROUP BY diagnosis""")
results = c.fetchall()
print(results)
print(round(results[1][1]-results[0][1],2))
c.close()

[('B', 12.15), ('M', 17.46)]
5.31


### 8) Change the name of the column "diagnosis" to "label"

In [12]:
c = conn.cursor()
c.execute("""ALTER TABLE bc RENAME COLUMN diagnosis TO label""")
conn.commit()
c.close()

### 9) Add a column named "day" of type DATE.

In [15]:
c = conn.cursor()
c.execute("""ALTER TABLE bc ADD COLUMN day DATE""")
conn.commit()
c.close()

### 10) Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)

In [17]:
c = conn.cursor()
c.execute("""ALTER TABLE bc
             ADD COLUMN area_mean_int INTEGER""")
c.execute("""UPDATE bc
             SET area_mean_int = CAST(area_mean AS INTEGER)""")
conn.commit()
c.close()

### 11) create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.

In [18]:
c = conn.cursor()

c.execute("""CREATE TABLE tomatch2(id INT PRIMARY KEY, rdm_float FLOAT)""")

query = """INSERT INTO tomatch2(id, rdm_float) VALUES (?,?)"""

list_of_insertion = []
for i in tqdm.tqdm(range(50000,10000000,10)):
    values = (i,rd.random())
    list_of_insertion.append(values)
    if len(list_of_insertion) == 10000:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()
c.close()

100%|██████████| 995000/995000 [00:04<00:00, 207086.39it/s]


### 12) Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2.

In [21]:
c = conn.cursor()
c.execute("""SELECT * FROM bc
             INNER JOIN tomatch ON bc.id = tomatch.id 
             INNER JOIN tomatch2 ON tomatch.id = tomatch2.id
             where radius_mean > 15 AND tomatch.rdm_float > 0.5 AND tomatch2.rdm_float > 0.5""")
c.close()

## MySQL

### 1) How many orders were placed in total

In [10]:
import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="m1_ds2e"
)

c = mydb.cursor()

c.execute("SELECT * FROM sample_superstore")

docs = c.fetchall()

columns = [description[0] for description in c.description]
df = pd.DataFrame(docs, columns=columns)
df.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,5/28/2012,5/30/2012,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/7/2010,7/8/2010,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,7/27/2011,7/27/2011,998.2023,8,1446.67,88523


In [8]:
c.execute("SELECT count(distinct `Order ID`) from sample_superstore")
c.fetchall()

[(127,)]

### 2) What is the total sales amount ?

In [11]:
c.execute("SELECT sum(Sales) from sample_superstore")
c.fetchall()

[(130555.31000000006,)]

### 3) What is the average profit per order?

In [15]:
c.execute("""SELECT `Order ID`, avg(Profit) as moy 
          FROM sample_superstore 
          GROUP BY `Order ID`""")
c.fetchall()

[(88525, 1.32),
 (88522, 4.56),
 (88523, 306.6841),
 (88524, 1388.0523),
 (88526, 1001.4453),
 (90193, 4390.3665),
 (90197, -141.26),
 (90194, 1045.4673),
 (90200, 406.7419),
 (90198, 72.9928),
 (90199, -252.67759875000002),
 (90195, 142.7955),
 (90192, -53.8096),
 (90196, 23.12),
 (86838, 185.425125),
 (86845, -44.54),
 (86837, -59.82),
 (86839, 261.8757),
 (86840, -1.17),
 (86841, -24.03),
 (86842, 23.2668),
 (86843, -174.2696),
 (86836, 327.68155),
 (86844, 3.4595),
 (90033, -6.82),
 (90034, -96.33),
 (90031, 314.4813),
 (90035, 164.48680000000002),
 (90038, -17.49),
 (90039, 139.54835),
 (90032, -114.6399),
 (90036, -491.73199999999997),
 (90037, -41.82),
 (41793, 384.38),
 (42949, -66.06495),
 (55269, -41.82),
 (9895, -6.82),
 (17860, -96.33),
 (26887, 172.27274999999997),
 (50533, -491.73199999999997),
 (18596, 174.845),
 (87655, 571.89495),
 (87660, 83.24),
 (87669, 353.77216),
 (87651, -16.802),
 (87656, 1489.8411),
 (87662, -23.415),
 (87663, -88.102),
 (87657, -10.58),
 (8766

### 4) Which customer has placed the most orders ?

In [21]:
c.execute("""SELECT `Customer ID`
             FROM sample_superstore 
             GROUP BY `Customer ID`
             ORDER BY count(distinct `Order ID`) DESC
             LIMIT 1""")
c.fetchall()

[(21,)]

### 5) What is the total profit for each product category ?

In [22]:
c.execute("""SELECT `Product Category`, sum(Profit)
             FROM sample_superstore 
             GROUP BY `Product Category`""")
c.fetchall()

[('Office Supplies', 11605.983540000001),
 ('Technology', 10225.340091999999),
 ('Furniture', 1976.0725965000036)]

### 6) What is the average discount percentage ?

In [23]:
c.execute("""SELECT avg(Discount)
             FROM sample_superstore""")
c.fetchall()

[(0.0496571428571428,)]

### 7) Which region has the highest total sales ?

In [24]:
c.execute("""SELECT Region, sum(Sales)
             FROM sample_superstore
             GROUP BY Region""")
c.fetchall()

[('Central', 14261.619999999999),
 ('West', 75678.49999999999),
 ('East', 38033.84999999999),
 ('South', 2581.34)]

### 8) How many orders were shipped using each shipping mode ?

In [31]:
c.execute("""SELECT `Ship Mode`, count(distinct `Order ID`)
             FROM sample_superstore
             GROUP BY `Ship Mode`""")
c.fetchall()

[('Delivery Truck', 19), ('Express Air', 20), ('Regular Air', 100)]

### 9) What is the total quantity ordered for each product sub-category ?

In [33]:
c.execute("""SELECT `Product Sub-Category`, count(`Order ID`) as cnt
             FROM sample_superstore
             GROUP BY `Product Sub-Category`
             ORDER BY cnt DESC""")
c.fetchall()

[('Paper', 26),
 ('Pens & Art Supplies', 23),
 ('Telephones and Communication', 21),
 ('Appliances', 14),
 ('Office Furnishings', 13),
 ('Labels', 11),
 ('Office Machines', 10),
 ('Binders and Binder Accessories', 10),
 ('Chairs & Chairmats', 9),
 ('Storage & Organization', 8),
 ('Tables', 6),
 ('Rubber Bands', 6),
 ('Computer Peripherals', 6),
 ('Envelopes', 5),
 ('Scissors, Rulers and Trimmers', 3),
 ('Bookcases', 3),
 ('Copiers and Fax', 1)]

### 10) Which product has the highest profit margin ?

In [35]:
c.execute("""SELECT `Product Category`, avg(`Product Base Margin`) as pbm
             FROM sample_superstore
             GROUP BY `Product Category`
             ORDER BY pbm DESC
             LIMIT 1""")
c.fetchall()

[('Furniture', 0.5990322580645162)]

### 11) What is the total sales amount for each year ?

In [41]:
c.execute("""SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as annee, sum(`Sales`)
             FROM sample_superstore
             GROUP BY annee
             ORDER BY annee DESC""")
c.fetchall()

[(2013, 48236.200000000004),
 (2012, 44893.209999999985),
 (2011, 12048.119999999997),
 (2010, 25377.779999999995)]

### 12) Which customer segment has the highest average sales ?

In [42]:
c.execute("""SELECT `Customer Segment`, avg(`Sales`) as sls
             FROM sample_superstore
             GROUP BY `Customer Segment`
             ORDER BY sls DESC""")
c.fetchall()

[('Small Business', 1041.5648076923078),
 ('Home Office', 868.6964705882353),
 ('Consumer', 856.5270588235293),
 ('Corporate', 528.821797752809)]

## PostgreSQL

In [43]:
import pandas as pd

df = pd.read_csv("C:/Users/Ju/OneDrive/Documents/FSEG/M1/NoSQL/bank_full.csv")
df.head()

,age,job,marital,education,default_,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### 1) What is the average age of the customers ?

In [ ]:
import psycopg2
# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="new_db", user="postgres", password="root", host="localhost",  port="5050" )
cur = conn.cursor()
cur.execute("""SELECT avg(age) 
            FROM bank_full""")
conn.fetchall()

### 2) How many customers have defaulted on loans ?

In [ ]:
cur.execute("""SELECT count(*) 
            FROM bank_full 
            WHERE default_ = 'yes'""")
conn.fetchall()

### 3) What is the maximum balance in the account ?

In [ ]:
cur.execute("""SELECT max(balance) 
            FROM bank_full""")
conn.fetchall()

### 4) How many customers have a housing loan ?

In [ ]:
cur.execute("""SELECT count(*) 
               FROM bank_full 
               WHERE loan = 'yes' AND housing = 'yes'""")
conn.fetchall()

### 5) What is the education level of the youngest customer ?

In [ ]:
cur.execute("""SELECT education 
               FROM bank_full 
               ORDER BY age 
               LIMIT 1""")
conn.fetchall()

### 6) What is the marital status of the oldest customer ?

In [ ]:
cur.execute("""SELECT marital 
               FROM bank_full 
               ORDER BY age DESC 
               LIMIT 1""")
conn.fetchall()

### 7) What is the distribution of the months in which customers were last contacted ?

In [ ]:
cur.execute("""SELECT month, count(contact) 
               FROM bank_full 
               GROUP BY month""")
conn.fetchall()

### 8) What is the average duration of contact for customers who have a personal loan ?

In [ ]:
cur.execute("""SELECT avg(duration) 
               FROM bank_full 
               WHERE loan = 'yes' AND housing = 'no'""")
conn.fetchall()

### 9) How many customers were previously contacted, and what was the average duration of their last contact ?

In [ ]:
cur.execute("""SELECT count(contact) as NUMBER, avg(duration) as DURATION 
               FROM bank_full 
               WHERE loan != 'unknown'""")
conn.fetchall()

### 10) What is the distribution of job types among customers ?

In [ ]:
cur.execute("""SELECT job, count(*) 
               FROM bank_full 
               GROUP BY job""")
conn.fetchall()

### 11) How many customers have been contacted more than once in the current campaign ?

In [ ]:
cur.execute("""SELECT count(*) 
               FROM bank_full 
               WHERE previous > 1""")
conn.fetchall()

### 12) What is the success rate of previous marketing campaigns (poutcome) among customers

In [ ]:
cur.execute("""SELECT 
               (CAST((SELECT COUNT(*) FROM bank_full WHERE y='yes') AS FLOAT) / 
                CAST((SELECT COUNT(*) FROM bank_full) AS FLOAT)) 
               AS ratio""")
conn.fetchall()